In [3]:
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd

In [154]:
import numpy as np
import datetime
from datetime import datetime

In [ ]:
driver = webdriver.Safari(executable_path ='/usr/bin/safaridriver')
df = pd.DataFrame(columns=['time', 'temp', 'dew_point', 'humidity', 'wind', 'wind_speed', 'wind_gust', 'pressure', 'precip', 'condition'])

for i in range(1,13):
    days_in_month = 0
    if i in [1,3,5,7,8,10,12]:
        days_in_month = 32
    elif i in [4,6,9,11]:
        days_in_month = 31
    elif i in [2]:
        days_in_month = 29
    for j in range(1,days_in_month):
        date = f'{j}.{i}.2017'
        print(date)
        url = f'https://www.wunderground.com/history/daily/KMDW/date/2017-{i}-{j}'
        driver.get(url)
        time.sleep(5)
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        table = soup.find('table', attrs={'class': 'mat-table cdk-table mat-sort ng-star-inserted' })
        table_rows = table.find_all('tr')
        l = []
        for tr in table_rows:       
            td = tr.find_all('td')
            row = [tr.text for tr in td]
            l.append(row)
            l = list(filter(None, l))
        for k in l:
            df.loc[len(df)] = k
            

driver.quit()
      


In [50]:
df.tail()

,time,temp,dew_point,humidity,wind,wind_speed,wind_gust,pressure,precip,condition
10245,8:53 PM,6 °F,-5 °F,60 °%,NW,10 °mph,20 °mph,29.92 °in,0.0 °in,Fair
10246,9:53 PM,4 °F,-6 °F,64 °%,NW,10 °mph,0 °mph,29.92 °in,0.0 °in,Fair
10247,10:53 PM,4 °F,-8 °F,58 °%,WNW,12 °mph,0 °mph,29.94 °in,0.0 °in,Fair
10248,11:53 PM,2 °F,-8 °F,63 °%,WNW,7 °mph,0 °mph,29.95 °in,0.0 °in,Fair
10249,12:53 AM,1 °F,-11 °F,57 °%,WNW,12 °mph,0 °mph,29.95 °in,0.0 °in,Fair


In [51]:
df.to_csv('./data/weather_data.csv', index = False)

In [159]:
df = pd.read_csv('./data/weather_data.csv')

In [160]:
df['date'] = np.nan

In [161]:
first_hour = 0
last_hour = 24
day = 1

for i in range(1,13):
    days_in_month = 0
    if i in [1,3,5,7,8,10,12]:
        days_in_month = 32
    elif i in [4,6,9,11]:
        days_in_month = 31
    elif i in [2]:
        days_in_month = 29
    for j in range(1,days_in_month):
        r = range(first_hour,last_hour)
        df.loc[r,['date']] = [f'2017-{i}-{j}']
        first_hour = first_hour + 24
        last_hour = last_hour + 24

df.head()



,time,temp,dew_point,humidity,wind,wind_speed,wind_gust,pressure,precip,condition,date
0,1:53 AM,25 °F,18 °F,75 °%,W,3 °mph,0 °mph,29.33 °in,0.0 °in,Fair,2017-1-1
1,2:53 AM,25 °F,18 °F,75 °%,CALM,0 °mph,0 °mph,29.34 °in,0.0 °in,Fair,2017-1-1
2,3:53 AM,25 °F,19 °F,78 °%,SSW,5 °mph,0 °mph,29.36 °in,0.0 °in,Fair,2017-1-1
3,4:53 AM,24 °F,18 °F,77 °%,SSW,7 °mph,0 °mph,29.37 °in,0.0 °in,Fair,2017-1-1
4,5:53 AM,22 °F,17 °F,82 °%,SSW,6 °mph,0 °mph,29.39 °in,0.0 °in,Fair,2017-1-1


PROBLEM: Es gibt nicht für jeden Tag exakt 24 Einträge (also einen pro Stunde). Es wurde immer um xx:53 daten erhoben, manchmal aber auch öfter. Daher müssen alle Einträge in time welche nicht mit :53 enden deleted werden und dann kann das Datum als eigene Column hinzugefügt werden mit dem obigen Code. 

In [146]:
df['date'] = pd.to_datetime(df['date'], format = '%Y-%m-%d')

In [182]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10250 entries, 0 to 10249
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   time        10250 non-null  string
 1   temp        10250 non-null  object
 2   dew_point   10250 non-null  object
 3   humidity    10250 non-null  object
 4   wind        10243 non-null  object
 5   wind_speed  10250 non-null  object
 6   wind_gust   10250 non-null  object
 7   pressure    10250 non-null  object
 8   precip      10250 non-null  object
 9   condition   10250 non-null  object
 10  date        8760 non-null   object
dtypes: object(10), string(1)
memory usage: 881.0+ KB
